![air-paradis](https://drive.google.com/uc?id=1T26mpOAUvJP700W4m8bjfYCLmDYVcyJL)

# <font color=red><center>**AIR PARADIS**</center></font>

**Air Paradis** is an airline company that wants to use AI (*Artificial Intelligence*) to **detect Bad Buzz associated with its brand** in online public tweets.

**As AI engineer for Marketing Intelligence Consulting**, we will dive into **NLP** (*Natural Language Processing*) techniques to serve Air Paradis' purpose.

Indeed, NLP allows a machine to **understand and process human language**. It will help us to solve this **text classification goal** and **detect sentiment** (positive or negative) from these tweets.

We will deploy our best **DETECT SENTIMENT solution** through <font color=salmon>**Microsoft Azure Machine Learning plateform**</font> (***MS Azure ML***).

<br>

Therefore, we will structure the project as follows:

<br>

| **Services / Tools** | **Objective** | **Available notebook** |
| :-- | :-- | :-- |
| **Google Colab and Python libraries** | Build quality of data by pre-processing the tweets text | Notebook N°1 |
| **Google Colab / MS Azure Cognitive Services API** | Use Text Analytics > Sentiment API | Notebook N°2 |
| **Python Script / MS Azure ML Studio > Designer** | Use "Drag-and-Drop" pipeline with no code in Azure ML Studio| Notebook N°3 |
| **Tensorflow-Keras / Google Colab PRO with GPU/TPU** | Train and evaluate advanced models | Notebook N°4 |
|**MS Azure ML Cloud > Models**| Deploy the best solution in MS Azure WebService | **<font color=green>Notebook N°5</font>** |

<br>

This notebook is dedicated to 5th task : **deploy our best model as a web service in the Azure cloud for Air Paradis**.

# <font color=brown><center>**NOTEBOOK 5<br>MODEL DEPLOYEMENT AS WEB SERVICE<br>AZURE CLOUD**</center></font>

The **workflow** is as follows:
- Register the model;
- Prepare an entry script;
- Prepare an inference configuration;
- Prepare a deployment configuration;
- Deploy the model;
- Test the resulting web service.

The details are available on MS Azure ML page [here](https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-deploy-and-where?tabs=python).

***Prerequisites***:
- Azure Machine Learning workspace;
- Azure Machine Learning SDK for Python (Software Development Kit);
- A folder with model (and tokenizer file for our case);
- A requirement.text for pip or Conda Dependencies.

# <font color=salmon>INSTALL AZURE ML SDK</font>

First, we install the **Azure ML SDK (*Software Development Kit*)** for Python.

The Azure ML SDK for Python is used by data scientists and AI developers to build and run machine learning workflows upon the Azure Machine Learning service.

We can interact with the service in any Python environment (Jupyter Notebooks, Google Colab or any Python IDE).

In [1]:
from IPython.display import clear_output

# Install azure ml SDK
!pip install azureml-core

clear_output()

In [2]:
import azureml.core

# Check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.27.0


# <font color=salmon>CONNECT TO WORKPLACE</font>

The **Workspace** is the top-level resource in Azure Machine Learning.

It allows to manage machine learning artifacts like environments, data stores, models, experiments or compute targets.

The workspace is tied to an Azure subscription and resource group, and supports region affinity.

In [3]:
from azureml.core import Workspace

# Connect to workspace
ws = Workspace.from_config('/content/drive/MyDrive/OC_IA/P07/p7_05_ws_config.json')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F7UPDF6D2 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


![connect](https://drive.google.com/uc?id=1CBBxpf32lL4sfaOpZIUtgUWXTe_jNktm)

# <font color=salmon>REGISTER THE MODEL</font>

When we register a model, we **upload it to the cloud** (in our workspace's default storage account) and then mount it to the same compute where our webservice is running.

In [4]:
from azureml.core.model import Model

# Register a model
model = Model.register(workspace = ws,
                       model_path= '/content/drive/MyDrive/OC_IA/P07/deploy_model', # include all the files in the folder
                       model_name = 'tweet_sentiment_glove_lstm',
                       description = 'Sentiment analysis with Glove embeddings trained outside AML')

Registering model tweet_sentiment_glove_lstm


We can check the model creation on Azume Machine Learning Studio in the <code>**Models**</code> section.

![model](https://drive.google.com/uc?id=1TFbhWVrGya_vXP6mYFy7EeUantcuJ8WO)

# <font color=salmon>WRITE ENTRY SCRIPT</font>

Here we write the **entry script (*score.py*)** that will be used to deploy and predict with our model, including the following 2 main parts:
- Load model with <code>**init()**</code> function;
- Run model on input data with <code>**run()**</code> function.

These are used to **initialize service** when the model is started, as well as **run the model** on data provided by the client. The other parts of the script take care of loading and running the model.

There is no universal script for all models. We must create a script that specifies how the model loads, what kind of data it expects, and how the model is used to evaluate data.

Other functions can be added as helpers.

In [5]:
%%writefile score.py

import numpy as np
import json
import pickle
from time import time
import os

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


def init():
    global glove_model
    global tokenizer
    
    # Get the path where the deployed model can be found
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'deploy_model')
    
    # Load existing model
    glove_model = load_model(model_path + '/glove_model.h5')
    
    # Load tokenizer
    with open(model_path + '/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

# Handle request to the service
def run(data):
    try:
        # Pick out the text property of the JSON request
        # Expected JSON details {"text": "some text to score for sentiment"}
        data = json.loads(data)
        prediction = predict(data['text'])
        return prediction
    except Exception as e:
        error = str(e)
        return error

# Determine sentiment from score
NEGATIVE = 'NEGATIVE'
POSITIVE = 'POSITIVE'
def decode_sentiment(score):
    return NEGATIVE if score < 0.5 else POSITIVE


# Predict sentiment using the model
SEQUENCE_LENGTH = 36
def predict(text):
    start = time()
    
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]),
                           maxlen=SEQUENCE_LENGTH)
    
    # Predict
    score = glove_model.predict([x_test])[0]
    
    # Decode sentiment
    label = decode_sentiment(score)

    return {'label': label, 'score': float(score),
       'elapsed_time': time()-start}  

Writing score.py


# <font color=salmon>PREPARE ALL DEPLOYMENT CONFIGURATIONS</font>

## <font color=green>INFERENCE CONFIGURATION</font>

The inference configuration specifies an **environment** including the **dependencies** that enables the deployment of our model, and the **scoring script** that will be used for inference.

In [6]:
from azureml.core.environment import Environment

# Name environment and call requirements file
# requirements: numpy, tensorflow, azumeml-defaults
myenv = Environment.from_pip_requirements(name = 'myenv',
                                          file_path = '/content/drive/MyDrive/OC_IA/P07/requirements.txt') 

In [7]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

# Create inference configuration
inference_config = InferenceConfig(environment=myenv,
                                   entry_script='score.py')

## <font color=green>DEPLOYMENT CONFIGURATION</font>

Deploy the model means **convert it into an API** so users can call it and make predictions.

We can choose to run the model as:
- a local instance (LocalWebservice) for development purposes;
- an Azure Container Instance (ACI) for Q&A (question and answer) testing purposes;
- an Azure Kubernetes Service (AKS) for production use.

Our choice is to deploy that model to **Azure Container Instances (ACI)**.

ACI is suitable only for small models (otherwise, recommendation is to use single-node AKS to dev-test larger models).

In [8]:
from azureml.core.webservice import AciWebservice #AksWebservice

# Set the virtual machine capabilities
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 0.5,
                                                       memory_gb = 3)

# <font color=salmon>DEPLOY THE MODEL</font>

To deploy our web service, we need to combine our environment, our inference compute, our scoring script and our registered model in the method <code>**deploy()**</code>.

---

This service can have several ***states***:
- <font color=orange>**Transitioning**: the service is in the process of deployment - not a final state</font>;
- <font color=orange>**Unhealthy**: the service had deployed but is currently unreachable - not a final state</font>;
- <font color=orange>**Unschedulable**: the service cannot be deployed at this time due to lack of resources - not a final state</font>;
- <font color=red>**Failed**: the service had failed to deploy due to an error or crash - final state</font>;
- <font color=green>**Healthy**: the service is healthy and the endpoint is available - final state</font>.

---

**The goal is Healthy state!!**

## <font color=green>RUN THE DEPLOYMENT</font>

When we deploy the model, the **Azure Container Registry** (ACR) is created and this is one of the priced services.

In [9]:
from azureml.core.model import Model

# Deploy ML model (Azure Container Instances)
service = Model.deploy(workspace=ws,
                       name='text-sentiment-service',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config)

service.wait_for_deployment(show_output = True)

# State should be healthy for successful deployment
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-06 21:05:48+00:00 Creating Container Registry if not exists.
2021-05-06 21:05:48+00:00 Registering the environment.
2021-05-06 21:05:49+00:00 Use the existing image.
2021-05-06 21:05:49+00:00 Generating deployment configuration.
2021-05-06 21:05:51+00:00 Submitting deployment to compute..
2021-05-06 21:06:25+00:00 Checking the status of deployment text-sentiment-service..
2021-05-06 21:08:38+00:00 Checking the status of inference endpoint text-sentiment-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


On Azure ML Studio, we can see the ACR creation in the <code>**Endpoints**</code> section.

![acr](https://drive.google.com/uc?id=1BPfdzjO4intPmonFmCiHPOIrkW-PoMIZ)

When the deployment is successful, we can see the State in Azure ML Studio as **Healthy** and the **REST endpoint** is available to consume the service.

![healthy](https://drive.google.com/uc?id=1ZBB8QzoHWBz-mWbzfXIPfdIZHBMd5zwr)

## <font color=green>CHECK DEPLOYMENT STATUS</font>

We can check the service logs, especially if the service is **not healthy** or if we experience errors.

In [17]:
# View the service logs
print(service.get_logs())

2021-05-06T20:42:58,618827400+00:00 - iot-server/run 
2021-05-06T20:42:58,622614200+00:00 - rsyslog/run 
2021-05-06T20:42:58,623876500+00:00 - gunicorn/run 
2021-05-06T20:42:58,646396900+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_d19f757c2fdd83e0b05731ace6695b70/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_d19f757c2fdd83e0b05731ace6695b70/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_d19f757c2fdd83e0b05731ace6695b70/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_d19f757c2fdd83e0b05731ace6695b70/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_d19f757c2fdd83e0b05731ace6695b70/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

These logs are also available on Azure ML Studio.

![logs](https://drive.google.com/uc?id=1R4_g0FuVVS0me0ocTOcgQbh2nafTqe_G)

## <font color=green>VIEW FROM AZURE PORTAL'S GRAPHICAL INTERFACE (STUDIO)</font>

# <font color=salmon>CONSUME THE WEB SERVICE</font>

After deploying the service, we can consume it from client applications to predict sentiments for new data cases.

To do so, we grab the **scoring URI** for our newly deployed model. It's this scoring URI that our clients can make POST requests to, in order to make predictions against our model.

The input data is a text in JSON format: it will be put into the body of the HTTP request and sent to the service encapsulating the model for scoring.

In [13]:
import requests
import json

# Test after deployment
# Set environment variables
scoring_uri = 'copy the REST endpoint here' # this need to be fulfilled
headers = {'Content-Type': 'application/json'}

# Provide a text example
data = json.dumps({'text':'user that is a bummer url hashtag'})

# Call with POST request
response = requests.post(scoring_uri, data=data, headers=headers)

# Print result
print('Status code: ', response.status_code)
print('This tweet is: ', (response.json()).get('label'))
print('Its score is: ', (response.json()).get('score'))
print('Elapsed time: ', (response.json()).get('elapsed_time'))

Status code:  200
This tweet is:  NEGATIVE
Its score is:  0.035684049129486084
Elapsed time:  0.0719151496887207


The service can also be tested on Azure ML Studio.

![test](https://drive.google.com/uc?id=1wNM1GT82CKliCZeYevtW_O4uS6Ifh8qn)

# <font color=salmon>EXPORT MODEL</font>

We can download a register model by navigating to the desired **Model** and choosing **Download**.

![download](https://drive.google.com/uc?id=1e6pVBpL8t73WHM4_RJuzz0OCC--lPfNU)

# <font color=salmon>DELETE UNUSED RESOURCES</font>

A (compute) instance does not automatically scale down, so we need to make sure to stop the resource to prevent ongoing charges.

In [14]:
# Delete the (web) service
service.delete()

In [15]:
# Delete the model
model.delete()

Then we delete the Azure Container Registry in Azure Portal.

![delete](https://drive.google.com/uc?id=1jCNa-on-5poX09v6ex0z3QcEbpFmBY90)